## 第二版本 后端全流程

### Web多应用综合，初步完成demo

#### 目的，demo，调试日志，多层次测试，手动构建与生产部署
  


  - 业务核心功能
    - 熟悉mysql,redis，mongodb及其对应异步驱动，为之后扩展迁移准备
    - 核心业务梳理，模块划分
    - 数据库设计
    - MySQL,MongoDB,Redis异步操作
    - 数据的CUDR服务
    - 路由接口设计
    - 认证授权
    - 异常处理
    - 安全防护
      - 防SQL注入
      - XSS
      - CSFR
    - 性能优化
      - 缓存
      - 异步任务
      - 负载均衡
      - 多进程多线程
  - 调试和日志信息
  - 单元测试，集成测试，端点测试，负载测试
  - 简单手动镜像构建和生产部署


文件组织
  - backend
    - users
      - databases 数据模型
      - models  表模型
      - schemas 表模型前数据模型
      - services 表模型的CUDR
      - api   和其他模块接口
      - routers  路由接口
      - static
      - tests   单元测试
      - utils   工具
      - config.py  模块配置，可从env读取，或直接写死 
      - dependencies.py
      - exceptions.py
    - blogs
      - xx
    - chats
      - xx
    - forum
      - xx
    - shops
      - xx
    - logs
      - users_logs
        - xx-xx-xx
    - tests集成测试
      - xx
    - app.py
    - main.py
    - .env
    - test.ipynb
  - 项目依赖和环境管理

核心功能点:
1. 用户注册只能通过邀请码
2. 用户主账号可以关联两个匿名账号
3. 朋友关系可达6度（朋友的朋友的朋友...最多6层），并且可以查看关系链
4. 用户之间可以设置熟悉等级（陌生，了解，熟悉，信任）
5. 用户可以关注其他用户
6. 用户可以写博客（支持标签、草稿、搜索、评论回复），并且评论回复可以转化为帖子
7. 用户可以对博客标记喜欢、不喜欢、收藏
8. 用户之间可以建立私聊或群聊，聊天内容可以转化为帖子
9. 用户可以直接创建帖子(支持标签，用户回复),可对帖子标记喜欢，不喜欢，收藏

#### 用户应用

- 根据邀请码，注册用户
- 用户主账号可以关联两个匿名账号。
- 朋友关系可达6度（朋友的朋友的朋友...最多6层），并且可以查看关系链。
- 用户之间可以设置熟悉等级（陌生，了解，熟悉，信任）。
- 用户可以关注其他用户。

MySQL数据库设计（用于存储结构化数据，如用户信息、关系、邀请码等）
- 用户表users
    - id: 主键
    - serial: 用户序列（唯一）
    - username: 用户名（非唯一）
    - type: anonymous|primary
    - email: 邮箱（非唯一）
    - phone：手机（非唯一）
    - password_hash: 密码哈希
    - salt: 椒盐值
    - created_at: 创建时间
    - last_login: 最后登录时间
    - status: 状态（在线、禁用、隐身）
- 用户权限表user_privileges
    - id
    - user_id
    - invitation_code_num
    - create_invitation
    - anonymous_num
    - create_anonymous
    - relation_gap
- 主账号与匿名账号关联表user_anonymous_accounts
    - id: 主键
    - primary_user_id: 主账号ID（外键到users.id）
    - anonymous_user_id: 匿名账号ID（外键到users.id）
    - created_at: 创建时间
- 邀请码表invitation_codes
    - code: 邀请码（主键）
    - created_by: 创建者（可能是管理员或用户，如果允许用户生成邀请码，则记录；否则可以是系统）
    - created_at: 创建时间
    - used_by: 被哪个用户使用（外键到users.id，可为空）
    - used_at: 使用时间
- 用户关系表user_relationships，仅记录gap=0的关系
    - id: 主键
    - user_id: 用户ID（外键到users.id）
    - related_user_id: 相关用户ID（外键到users.id）
    - level: 熟悉等级（0:陌生, 1:了解, 2:熟悉, 3:信任）
    - created_at: 关系创建时间
    - updated_at: 最后更新时间
- 关注表follows
    - follower_id: 关注者ID
    - followed_id: 被关注者ID
    - created_at
    - 主键 (follower_id, followed_id)


Redis数据库设计
- 关系缓存


注意事项：
- 匿名账号也有朋友圈等功能，完全和主账号一样
- 匿名账号和主账号可以互相访问对方的资源
- 主账号为了信任机制，必须填写正确内容，匿名帐号没限制
- 匿名账号可以销毁和重建，前期可以设定数量
- 每个人的邀请码也可以限制数量
- 邀请注册，邀请码和serial别人看不见
- 朋友的定义，认为是双向都同意(可以实现沟通)，才能创建，并必须附带关系等级
- 邀请码默认朋友，朋友相互点亮(和相互关注解耦)，关系等级
- gap=设置值的关系链，在redis中查看，避免重复计算
- 用户还有职业，标签，项目，技能，经历等信息，方便日后做搭配使用，现在先不设计，既可以mysql设计多个单表，也可以用mongodb存储半结构数据，方便扩展

#### 博客应用

- 用户可以写博客，支持标签，草稿，搜索，评论回复
- 评论回复可以转换成帖子
- 用户可以对博客标记喜欢，不喜欢，收藏

MySQL数据库设计
- 博客表blogs
    - id: 主键
    - user_id: 作者ID
    - title: 标题
    - content: 内容（文本，或者存储为文本，大文本）
    - status: 状态（草稿、发布、私有等）
    - created_at: 创建时间
    - updated_at: 更新时间
- 标签表tags
    - id: 主键
    - name: 标签名（唯一）
- 博客标签关联表bog_tags
    - blog_id
    - tag_id
    - 主键 (blog_id, tag_id)
- 博客操作表blog_actions
    - id
    - user_id
    - blog_id
    - likes
    - favorites
- 评论表comments
    - id: 主键
    - user_id: 评论者ID
    - blog_id: 博客ID（如果是对博客的评论）
    - parent_comment_id: 父评论ID（如果是回复评论）
    - content: 评论内容
    - created_at
    - updated_at
    - is_post: 是否已经被转换成了帖子
- 举报表
    - user_id
    - blog_id
    - dislikes：设置后，自动取消喜欢和收藏
    - report

#### 聊天应用

- 用户之间可以建立私聊或群聊
- 聊天内容可以转换成帖子

MySQL数据库设计
- 会话表conversations
    - id: 主键
    - type: 类型（private/group）
    - name: 群聊名称（如果是私聊则为空）
    - creator_id: 群聊创建者（私聊则为空）
    - created_a
- 会话成员表chat——members
    - conversation_id
    - user_id
    - joined_at: 加入时间
    - 主键 (conversation_id, user_id)
- 消息表messages
    - id: 主键
    - conversation_id: 会话ID
    - user_id: 发送者
    - content: 消息内容
    - created_at
    - is_post

Mongodb数据库设计

注意事项：
- Mongodb暂存，转换到Mysql,后面熟悉了，逐步过渡到Mongodb

#### 论坛应用

- 用户可以直接创建帖子，支持标签，用户回复
- 可对帖子标记喜欢，不喜欢，收藏

MySQL数据库设计
- 帖子表posts
    - id: 主键
    - user_id: 发帖人ID
    - title: 标题（可能为空，因为由评论转化的帖子可能没有标题）
    - parent：上一个帖子，被转换的也有
    - content: 内容
    - source_type: comment|chat|direct
    - source_id
    - created_at
    - updated_at
- 帖子标签表tags，不用论坛的分类，用标签，多对多，更灵活
- 帖子标签关联表post_tags
- 帖子操作表post_actions

## 第三版本 多端前端

### Vue项目设计及学习


不同颗粒度原型设计，对产品功能、界面结构、交互等方面进行定义
  - 粗颗粒度
    - 产品的整体框架和核心功能
    - 不关注细节，快速验证产品和基本概念和方向
    - 手绘草稿，简单工具
    - 初步沟通，确认产品方向，快速迭代，探索多种可能性
  - 中颗粒度
    - 增加了更多细节，包括页面的布局，交互设计，部分视觉元素
    - 详细布局：明确页面布局，控件摆放，页面之间的跳转关系
    - 交互设计：包含基本的交互逻辑，如按钮点击，表单提交等
    - 视觉初步：包含简单的视觉设计元素，如颜色，字体等，但不追求整体的效果
    - 交互流程图：详细展示页面之间跳转逻辑和交互细节
    - 应用场景：产品设计阶段，详细规划产品的功能和交互，明确开发需求，用户测试并收集反馈
  - 细颗粒度
    - 每一个细节精心设计，包括完整的视觉设计，交互细节，动画效果等
    - 完整视觉：完整的视觉设计，包括颜色，字体，图标等，与最终产品一致
    - 精细交互：包含所有交互细节，如动画效果，过渡效果等
    - 高度保真：几乎与最终产品一致，用于用户测试和验收
    - 产品开发阶段，用于知道开发人员进行界面开发
    - 用户测试阶段，用于收集用户对最终产品的反馈
    - 产品验收阶段，用于确认产品是否符合设计要求
    - 高保真原型工具：Figma，Sketch等
    - 可交互的动画原型工具：Principle，Framer等

功能，模块和文件组织划分
- api请求模块化
  - moduleA.js
  - moduleB.js
- 基础组件components
  - common 公共组件, Element Plus组件的二次封装，Bootstrap组件二次封装
  - 模块A 模块专用组件
  - 模块B
- 工具组件
- 页面组件
  - 模块A
    - Page1.vue
    - Page2.vue
  - 模块B
    - Page1.vue
    - Page2.vue
- 路由配置
- 状态管理
- 全局样式
- 测试文件
- 根组件
- 应用入口

学习体验
- Axio
- Mock.js

### 项目demo

大屏网页

桌面应用

移动应用

移动网页

## 第四版本 算法开发